In [1]:
# IMPORT ALL RELEVANT LIBRARIES

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
import pandas as pd
import time
import re

In [3]:
### DRIVER SETUP

service = Service(r"C:\Users\MattiaDevescovi\Desktop\chromedriver.exe")
driver = webdriver.Chrome(service=service)

# open up a specific web page
driver.get("https://www.farmae.it/")

In [4]:
# Creation of the list of links of pages that needs to be scraped
n = int(input("Enter a number: "))
number_of_pages = list(range(32, n+1))
pagination = "https://www.farmae.it/erboristeria.html?p="
link_list = [pagination + str(page) for page in number_of_pages]

In [5]:
for index_link, link in enumerate(link_list):
    driver.get(link)
    time.sleep(3)
    WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.XPATH, "//a[@class='product-item-link']")))

    ## Create a list with all the product links
    html_tag = driver.find_elements(By.XPATH, '//a[@class="product-item-link"]')
    html_prod_links = [l.get_attribute('outerHTML') for l in html_tag]
    prod_links = []
    for link in html_prod_links:
        url = re.findall('href="(.*?)"', link)
        prod_links.append(url[0])
        time.sleep(2)

    time.sleep(30)

    ## Start iterating on every product
    for index_products, url_products in enumerate(prod_links):
        urls_list = "'"+url_products+"'"
        driver.get(url_products)

        # ERROR 404
        product_image_error_tags = driver.find_elements(By.XPATH, '//*[@id="maincontent"]/div[2]/div/div[4]/div/div/div[1]/figure/a/img[2]')
        if len(product_image_error_tags) > 0:
            error_image = product_image_error_tags[0].get_attribute('outerHTML')
            if 'banner-404-20220913-it.jpg' in error_image:
                continue

        WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.XPATH, "//span[@class='base']")))
        time.sleep(4)

        # creation of a variable last_index so that I can stack all the dataframes of each page
        last_index = len(prod_links)
        
        # PRODUCT ID (MINSAN)
        product_id_tag = driver.find_element(By.XPATH, "//h3[@itemprop='sku']")
        product_id = product_id_tag.text

        # PRODUCT NAME
        product_name_tag = driver.find_element(By.XPATH, "//h1[@class='page-title']")
        product_name = product_name_tag.text

        # PRODUCT DESCRIPTION
        product_description_tag = driver.find_element(By.XPATH, "//div[@class='brand-description']")
        product_description = product_description_tag.text
        product_description

        # PRODUCT CATEGORIZATION
        product_SC_tag = driver.find_elements(By.XPATH, "//span[@class='category']")
        text_list = [product_SC_tag.text for product_SC_tag in product_SC_tag]
        # create two columns containing the second and third elements of the list
        Super_Collection = []
        Collection = []
        Section = []
        try:
         Super_Collection.append(text_list[0])
        except IndexError:
         Super_Collection.append('')
        try:
         Collection.append(text_list[1])
        except IndexError:
         Collection.append('')
        try:
         Section.append(text_list[2])
        except IndexError:
         Section.append('')

        # PRODUCT IMAGE
        product_image_tag = driver.find_element(By.TAG_NAME, "img")
        html_image = product_image_tag.get_attribute('outerHTML')
        match = re.search('src="(.*?)"', html_image)
        image_url = []
        if match:
            image_link = match.group(1)
            image_url.append(image_link)
            
        ### STRUCTURE SUBCATEGORY INTO A DATAFRAME
        if index_products == 0:
            pharma_master_catalogue = pd.DataFrame(
                {'external_ID': product_id,
                'Super Collection': Super_Collection,
                'Collection': Collection,
                'Section': Section,
                'name': product_name,
                'description': product_description, 
                'image': image_url
                })
        if index_products != 0:
            temp_pharma_master_catalogue = pd.DataFrame(
                {'external_ID': product_id,
                'Super Collection': Super_Collection,
                'Collection': Collection,
                'Section': Section,
                'name': product_name,
                'description': product_description, 
                'image': image_url
                })
            data = [pharma_master_catalogue, temp_pharma_master_catalogue]
            pharma_master_catalogue = pd.concat(data)
        if index_products == last_index - 1:
            if index_link == 0:
                partner_menu = pharma_master_catalogue
            if index_link != 0:
                data = [partner_menu, pharma_master_catalogue]
                partner_menu = pd.concat(data)
        
        time.sleep(4)
    
    # Save the dataframe to csv after every iteration of link_list
    filename = f"menu_farmaè_pg_{index_link}.csv"
    path_csv = (r"C:\Users\MattiaDevescovi\Desktop\Scraping farmaè\\")
    partner_menu.to_csv(path_csv + filename, index=False)
